In [ ]:
#r "nuget: Microsoft.SemanticKernel"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using System.IO;

public class Architecture
{
    public string Request { get; set; }
    public string Response { get; set; }
} 
 
static string ParseFile(string filePath)
{
    var fileContent = File.ReadAllText(filePath);

        // Splitting the content into Request and Response
        var sections = fileContent.Split(new string[] { "#Request", "#Response" }, StringSplitOptions.RemoveEmptyEntries);
        if (sections.Length >= 2)
        {
            var setup = new Architecture
            {
                Request = sections[0].Trim(),
                Response = sections[1].Trim()
            };

            // Serialize to JSON
        return JsonSerializer.Serialize(setup);
    }
    else
    {
        throw new Exception("Invalid file format");
    }
}

In [ ]:
var files = Directory.GetFiles(".\\data\\eraser-samples");
var archs = files.Select(f => ParseFile(f)).ToList();

In [ ]:
var eraserdiagram_template_promnpt = 
$"""
You are an expert in creating earaser diagrams. 
Please create a diagram that shows the process of creating an eraser diagram.
Make sure that all open open cury braces are closed with a closing brace.

For reference you can use the following samples:

## Samples:
{JsonSerializer.Serialize(archs)}
""";


In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");

var builder = Kernel.CreateBuilder();

builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var chat = new ChatHistory(eraserdiagram_template_promnpt);

In [ ]:
Func<string, Task> Chat = async (string input) => {
    chat.AddUserMessage(input);
    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());

    Console.WriteLine($"\nBot:");
    Console.WriteLine($"{assistantReply}\n");
};

In [ ]:
#pragma warning disable SKEXP0001

var user_prompt = """
I want to have a azure architecture diagram for a web application.
I want ot have a subscription with 2 ResourceGroups
In each ResourceGroup place a StaticWebApp called WebApp1, a storage account called Storage1 and a SQL Database called SQL1.
The AppService of each should have a connection to the SQL Database and the Storage Account in their ResourceGroup.
Add a third ResourceGroup called SharedResources with a KeyVault called KeyVault1.
All static web apps should have connection to the KeyVault.
Also create a FrontDoor ResourceGroup with an load balancer called FrontDoor1 which is connected to both StaticWebApps.
""";


await Chat(user_prompt);
